<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/Turkish/sub/sub1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Turkish Language

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install focal-loss

In [3]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Turkish/

/content/drive/My Drive/dataset/OffenseEval2020/data/Turkish


In [4]:
ls

cc.tr.300.vec     offenseval-annotation.txt     offenseval-tr-training-v1.tsv
cc.tr.300.vec.gz  offenseval-tr-testset-v1.tsv  readme-trainingset-tr.txt


In [0]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
train=pd.read_csv( 'offenseval-tr-training-v1.tsv',sep="\t")
train=train.sample(frac=1)

In [8]:
train.head()

,id,tweet,subtask_a
15408,30835,Hala ev arkadaşım olabilirsin tatlı kız ????? ...,NOT
8146,19332,"benim öfkem kendime,başa çıkmam muhtemel..",NOT
22178,15876,Gün bitimine simit ve ayranla yapılan güzel bi...,NOT
9933,38361,Lütfen çıkarlarınızı ve beklentilerinizi de al...,OFF
30523,37684,"@USER Trolcan yine işbaşında, sen bir Ensar va...",OFF


In [0]:
train["tweet"]= train["tweet"].str.lower()
train["tweet"]= train["tweet"].str.replace('@USER', "") 
train['tweet']=train['tweet'].str.replace('\d+', '')
train['tweet']=train['tweet'].str.replace('URL', '')

In [0]:
train_labels=train['subtask_a']
train=train['tweet']
labels=le.fit_transform(train_labels)

In [16]:
train.tail()

21896    dünkü ülke tv saçmalığından sonra asabım bozul...
21703    okuması yok, yazması var. aklı yok, fikri var....
24358    üç tişört iki pantolonum var  kombinasyon oluş...
12266     hayat porno değil..   bunun laik yaşamla alak...
6220     adamlar daha beni̇m peygamber olduğumu anlayam...
Name: tweet, dtype: object

In [18]:
labels[0:5]

array([0, 0, 0, 1, 1])

In [19]:
import collections
collections.Counter(labels)

Counter({0: 25231, 1: 6046})

In [20]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(train)
sequences = tokenize.texts_to_sequences(train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

Using TensorFlow backend.


In [21]:
len(sequences_matrix),len(labels)

(31277, 31277)

In [0]:
from keras.models import Sequential
from keras.layers import Embedding,CuDNNGRU,GRU,Dense,Dropout,Bidirectional,GlobalMaxPool1D,GlobalAveragePooling1D, SpatialDropout1D
from keras.optimizers import RMSprop
from keras.initializers import Constant
from sklearn.utils import class_weight
from sklearn.metrics import classification_report,f1_score
import keras
import numpy as np
from sklearn.model_selection import StratifiedKFold
from focal_loss import BinaryFocalLoss
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [23]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

10001


In [0]:
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tr.300.vec.gz

In [0]:
# import gzip
# import shutil
# with gzip.open('cc.tr.300.vec.gz', 'rb') as f_in:
#     with open('cc.tr.300.vec', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

In [0]:
embed_size=300
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def build_matrix(embedding_path, word_index):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

    nb_words = min(max_words, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [0]:
embeddings=build_matrix('cc.tr.300.vec', word_index)


In [28]:
embeddings.shape

(10001, 300)

In [0]:
from keras.optimizers import Adam
opt=Adam(0.0001)

In [0]:
def model_func():
  model = Sequential()
  model.add(Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False))
  model.add(Bidirectional(CuDNNGRU(50, return_sequences = True)))
  #model.add(Bidirectional(CuDNNGRU(32, return_sequences = True)))
  model.add(GlobalMaxPool1D())
  model.add(Dense(50, activation="relu"))
  model.add(Dense(20, activation="relu"))
  model.add(Dense(10, activation="relu"))

  model.add(Dropout(0.05))
  model.add(Dense(1, activation="sigmoid"))
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer=opt, metrics=['accuracy'])
  return model

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels),labels)
class_weights=dict(enumerate(class_weights))

In [0]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1)

In [46]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(sequences_matrix,labels,test_size=0.15,stratify=labels,shuffle=True)
model=model_func()
model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=2,class_weight=class_weights,validation_data=(x_val,y_val))

Train on 26585 samples, validate on 4692 samples
Epoch 1/10
 - 11s - loss: 0.1341 - acc: 0.7932 - val_loss: 0.1183 - val_acc: 0.8152
Epoch 2/10
 - 10s - loss: 0.1137 - acc: 0.8265 - val_loss: 0.1044 - val_acc: 0.8393
Epoch 3/10
 - 10s - loss: 0.1056 - acc: 0.8387 - val_loss: 0.1003 - val_acc: 0.8457
Epoch 4/10
 - 10s - loss: 0.1017 - acc: 0.8461 - val_loss: 0.0985 - val_acc: 0.8476
Epoch 5/10
 - 10s - loss: 0.1001 - acc: 0.8504 - val_loss: 0.0990 - val_acc: 0.8483
Epoch 6/10
 - 10s - loss: 0.0982 - acc: 0.8536 - val_loss: 0.0968 - val_acc: 0.8508
Epoch 7/10
 - 10s - loss: 0.0968 - acc: 0.8561 - val_loss: 0.0958 - val_acc: 0.8572
Epoch 8/10
 - 10s - loss: 0.0952 - acc: 0.8582 - val_loss: 0.0951 - val_acc: 0.8561
Epoch 9/10
 - 10s - loss: 0.0944 - acc: 0.8618 - val_loss: 0.0947 - val_acc: 0.8574
Epoch 10/10
 - 10s - loss: 0.0936 - acc: 0.8632 - val_loss: 0.0950 - val_acc: 0.8576


In [47]:
from sklearn.metrics import classification_report

y_pred = model.predict(x_val, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.5)

print(classification_report(y_val, y_pred))

4692/4692 [==============================] - 1s 151us/step
              precision    recall  f1-score   support

           0       0.88      0.95      0.92      3785
           1       0.71      0.45      0.55       907

    accuracy                           0.86      4692
   macro avg       0.79      0.70      0.73      4692
weighted avg       0.85      0.86      0.84      4692



In [49]:
test=pd.read_csv( 'offenseval-tr-testset-v1.tsv',sep="\t")
test["tweet"]= test["tweet"].str.replace('@USER', "") 
test['tweet']=test['tweet'].str.replace('\d+', '')
test['tweet']=test['tweet'].str.replace('URL', '')
test['tweet']=test['tweet'].str.lower()

test.head()

,id,tweet
0,41993,sayın başkanım bu şekilde devam inşallah👏
1,23000,"herkes gevşekliği kadar duyar kasıyor,hayat bö..."
2,42478,olgun ilişkisi olan arkadaş size en güzel hedi...
3,21748,burada atıp tutacağına o kötü koşullarda ku...
4,13607,i̇şte o onur dediğin sende yok sorun o işte


In [0]:
X_test=test['tweet']

test_sequences = tokenize.texts_to_sequences(X_test)
X_test_sequences = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [51]:
y_test = model.predict(X_test_sequences, batch_size=64, verbose=1)


3515/3515 [==============================] - 0s 125us/step


In [0]:
y_test[y_test>0.5]=1
y_test[y_test<0.5]=0
y_test=y_test.astype('int16').ravel()

In [53]:
y_test=le.inverse_transform(y_test)
y_test=pd.DataFrame(y_test,columns=['label'])
y_test=pd.concat([test['id'], y_test['label']], axis=1)
y_test.head()


,id,label
0,41993,NOT
1,23000,NOT
2,42478,NOT
3,21748,NOT
4,13607,NOT


In [0]:
y_test.to_csv('/content/sub1.csv',index=False,header=None)